# Funktionsobjekte und Lambda-Ausdrücke

#### Patrick Schnider, Marcel Lüthi</br>Departement Mathematik und Informatik, Universität Basel

In diesem Notebook werden wir das Konzept der funktionalen Programmierung streifen. Dabei werden wir zwei neue, nützliche Methoden für unsere Listenklasse einführen.

Die erste ist die Methode `map`.
Diese wendet auf jedem Element der Liste eine Operation an.
Zurückgegeben wird eine neue Liste, gefüllt mit den jeweiligen Resultaten.

Die zweite Methode heisst `filter`.
Filter wird genutzt um Elemente aus einer Liste auszusortieren.
Dabei wird eine neue, kleinere Liste erzeugt.
Diese enthält nur Elemente welche ein bestimmtes Prädikat erfüllen.
Anders gesagt, die verbleibenden Elemente erfüllen alle eine bestimmte Bedingung.

Welche Operation `map` auf jedem Element ausführt und welche Bedingung bei `filter` geprüft wird ist nicht in deren Implementation vorgegeben.
Dies kann durch den Benutzer der Methoden bestimmt werden.
Wir müssen also die Bedingung, respektive die Operation an die Methoden übergeben können.

#### Übung: 

* Diskutieren Sie, welches Sprachkonstrukt Sie dafür einsetzen könnten.

### Funktionen 

Um allgemeine Funktionen der Form
$$f: T \rightarrow R$$
zu implementieren können wir ein Interface `Function` schreiben.
Eine Funktion hat nur eine Methode, nämlich die Anwendung der Funktion auf einen Wert.
Wir nennen diese `apply`.

In [ ]:
interface Function<T, R> {
    R apply(T t);
}

#### Übung: 

* Schreiben Sie eine Funktion `Square` als Implementation des Interfaces `Function`, welche die Funktion $f(x)=x^2$ implementiert. Nehmen Sie für beide Typparameter `Integer`.

In [ ]:
class Square implements Function<Integer, Integer> {
    public Integer apply(Integer value) {
        return value * value;
    }
}

Wir können unsere Funktion nun verwenden.

In [ ]:
class SquareTest {
    public static void main(String[] args) {
        Square fun = new Square();
        int value = 2;
        System.out.println(
                value + " squared is " +
                fun.apply(value)
        );
    }
}

SquareTest.main(new String[0]);

### Implementation der `map` und `filter` Operation

Mit Hilfe dieser Abstraktion für Funktionen können wir nun die Methoden `map` und `filter` für unsere LinkedList-Implementation schreiben. 

#### Übung

* Ergänzen Sie das Interface um die Methoden `map` und `filter`. 

In [ ]:
interface List<E> {
    void add(E element);
    
    int size();
    
    E get(int index);
    
    <R> List<R> map(Function<E, R> operation);
    
    List<E> filter(Function<E, Boolean> condition);
    
}

Nachfolgend implementieren wir die beiden Methoden.

In [ ]:
class Node<E> {
    E value;
    Node<E> next;
    
    Node(E value) {
        this.value = value;
        this.next = null;
    }
}

In [ ]:
class LinkedList<E> implements List<E> {
    
    Node<E> first;
    Node<E> last;
    
    int size;
    
    // Erzeugt eine ArrayList mit gegebener Kapazität
    public LinkedList() {
        this.first = null;
        this.last = null;
        this.size = 0;
    }
    
    
    // Fügt ein neues Element am Ende der Liste an. 
    public void add(E element) {
        Node<E> newNode = new Node<E>(element);
        if (first == null) {
            first = newNode;
            last = newNode;
        } else {
            last.next = newNode;
            last = newNode;
        }        
        size = size + 1;
    }
    
    public int size() { 
        return size;
    }
    
    public E get(int index) {
        Node<E> curr = first;
        for (int i = 0; i < index; i = i + 1) {
            curr = curr.next;
        }
        return curr.value;
        
    }
    
    public <R> List<R> map(Function<E, R> operation) {
        List<R> newList = new LinkedList<R>();
        Node<E> current = first;
        while (current != null) {
            newList.add(operation.apply(current.value));
            current = current.next;
        }
        return newList;
    }
    
    public List<E> filter(Function<E, Boolean> predicate) {
        List<E> newList = new LinkedList<E>();
        Node<E> current = first;
        while (current != null) {
            if (predicate.apply(current.value)) {
                newList.add(current.value);
            }
            current = current.next;
        }
        return newList;
    }
    
    
   
     // Gibt die Liste aus
    @Override
    public String toString() { 
        if (first == null) {
            return "[]";
        } else {
            StringBuffer sb = new StringBuffer();
            sb.append("[");
            for (Node current = first; current != last; current = current.next) {
                sb.append(current.value);
                sb.append(",");
            }
            sb.append(last.value);
            sb.append("]");
            return sb.toString();
        }
    }
}

### Anwendung

Nun werden wir die beiden Methoden `map` und `filter` anwenden.
Dafür benötigen wir aber noch eine Funktion, welche wir als Prädikat für Filter verwenden können.

Wir verwenden die Funktion `EvenNumberPred`, welche ein Zahl `n` als Eingabe nimmt und `True` zurückgibt falls `n` gerade ist und sonst `False`.

In [ ]:
class EvenNumberPred implements Function<Integer, Boolean> {
    public Boolean apply(Integer e) {
        return e % 2 == 0;
    }
}

#### Übung
- Wenden Sie beide Methoden ```Square``` und ```EvenNumberPred``` einmal auf die Liste an.

In [ ]:
class MapFilterTest {
    public static void main(String[] args) {
        LinkedList<Integer> l = new LinkedList<Integer>();
        for (int i = 0; i < 10; i = i + 1) {
            l.add(i);
        }
        
        List<Integer> filteredList = l.filter(new EvenNumberPred());
        List<Integer> mappedList = l.map(new Square());
        System.out.println(filteredList);
        System.out.println(mappedList);
    }
}

MapFilterTest.main(new String[0]);

### Kurzschreibweise mit Lambdas 
Die Methoden `map` und `filter` sind sehr elegant.
Das Schreiben der Funktionen selbst, jeweils als Klasse, ist aber mühsam.
Java stellt mit Lambda-Funktionen hier eine kurze Schreibweise zur Verfügung. Für unsere Funktionen können wir eine sehr kompakte Schreibweise nutzen.

In [ ]:
import java.util.function.BiFunction;
class LambdaTest {
    public static void main(String[] args) {
    
        Function<Integer, Integer> fun = x -> x + 1;
        System.out.println(fun.apply(0));
    
        BiFunction<Integer, Integer, Integer> fun2 = (x, y) -> x + y;
        System.out.println(fun2.apply(3, 4));
    }
}

LambdaTest.main(new String[0]);

### Aufgaben

- Schauen Sie sich das Paket [`java.util.function`](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/function/package-summary.html) und die darin enthaltenen Interfaces an.
Finden Sie passende Interfaces zu unseren beiden Funktionen? Wie heissen diese?
- Übergeben Sie den Methoden `map` und `filter`, statt der erstellten Objekte, jeweils equivalente Lambda-Ausdrücke.

In [ ]:
class MapFilterTest {
    public static void main(String[] args) {
        LinkedList<Integer> l = new LinkedList<Integer>();
        for (int i = 0; i < 10; i = i + 1) {
            l.add(i);
        }
        List<Integer> filteredList = l.filter(x -> x % 2 == 0);
        List<Integer> mappedList = l.map(x -> x * x);
        System.out.println(filteredList);
        System.out.println(mappedList);
    }
}

MapFilterTest.main(new String[0]);